In [1]:
from datetime import datetime
from ib_insync import * 
import random
import math
import pytz
import ntplib
import time
import tests
import ccxt
import ccxt.pro as ccxtpro
import sqlalchemy
import pandas as pd
import numpy as np
from os.path import exists
from pykalman import KalmanFilter

username = 'postgres'  
password = 'proddb123'
host = '34.84.190.250' 
port = '5432'
database = 'prod' 

db_url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    username, password, host, port, database)

engine = sqlalchemy.create_engine(db_url)
cloud = engine.connect()

global_data_max_size = 250
spread_type = 'log'
entry_sigma=3
lookback=100
band_type = 'bollinger'
avg_type = 'ema'
average_type = 'ema'
bollinger_type = 'ema'
bollinger_vol_exp = 1

In [2]:
def get_prices_crypto_websocket(ticker, bar_size, bars, ohclv=False, return_all_data=False): #this is going to be the one we want to use next, and going to require some further editing 
    ticker = format_ticker(tick1=ticker)
    table = 'ohlcv_' + str(ticker).lower() + '_' + str(bar_size)
    if return_all_data == False: 
        res = pd.DataFrame(cloud.execute("SELECT close FROM " + str(table) + " ORDER BY timestamp desc").all())['close'].values.tolist()
        return res[:bars] #results are returned most recent bar first
    else:
        res = pd.DataFrame(cloud.execute("SELECT * FROM " + str(table) + " ORDER BY timestamp desc").all()).values.tolist()
        return res[:bars]
    
def format_ticker(tick1=None, ticker2=None, use_list=False, ticker_list=None):
    if tick1 and not ticker2: 
        if tick1[-4:] == 'USDT':
            return tick1.split('/')[0] + 'USDT'
        elif tick1[-4:] == 'BUSD':
            return tick1.split('/')[0] + 'BUSD'

    if use_list == False: 
        res = []
        if tick1[-4:] == 'USDT':
            res.append(tick1.split('/')[0] + 'USDT')
        elif tick1[-4:] == 'BUSD':
            res.append(tick1.split('/')[0] + 'BUSD')

        if ticker2[-4:] == 'USDT':
            res.append(ticker2.split('/')[0] + 'USDT')
        elif ticker2[-4:] == 'BUSD':
            res.append(ticker2.split('/')[0] + 'BUSD')
        return res[0], res[1]
    elif use_list == True: 
        adj_list = []
        for ticker in ticker_list:
            if ticker[-4:] == 'USDT':
                adj_list.append(ticker.split('/')[0] + 'USDT')
            elif ticker[-4:] == 'BUSD':
                adj_list.append(ticker.split('/')[0] + 'BUSD')
        return adj_list
    

def get_spread(prices_a=None, prices_b=None): 
    global global_data, hedge_ratio
    if prices_a == None or prices_b == None: 
        df = global_data['Spread'][::-1]
        return df.tolist()

    spread = []
    for i in range(len(prices_a)):
        if spread_type == 'log':
            spread.append(math.log(prices_a[i]) - hedge_ratio*math.log(prices_b[i]))
        elif spread_type == 'absolute':
            spread.append(prices_a[i] - hedge_ratio*prices_b[i])
    return spread

def get_data(spread=None):
    if spread == None:
        spread = get_spread()

    zscore = get_zscore(spread)
    band_up_df, band_down_df = get_bands(zscore) 
    z_moving_avg_df = get_moving_avg(zscore)

    if math.isnan(z_moving_avg_df.iloc[-1]) == True: 
        print('The duration we are looking back at is likely not longer than the lookback period! Fix to get correct functionality!')
        print(spread)
        print(len(spread))

    return zscore, band_up_df[0].tolist(), band_down_df[0].tolist(), z_moving_avg_df[0].tolist()
    
def get_zscore(spread):
    spread_df = pd.DataFrame(spread)
    moving_avg_df = get_moving_avg(spread, override='sma')
    std_df = spread_df.rolling(lookback).std()
    zscore = []
    for i in range(len(spread)):
        spd = float(spread_df.iloc[i])
        mva = float(moving_avg_df.iloc[i])
        std1 = float(std_df.iloc[i])
        zscore.append((spd-mva)/std1)
    return zscore

def get_bollinger_bands(zscore, sigma=entry_sigma):
    zscore_df = pd.DataFrame(zscore)
    moving_avg = get_moving_avg(zscore)
    if bollinger_type == 'ema':
        std = zscore_df.ewm(span=lookback).std()
    else: 
        std = zscore_df.rolling(lookback).std()
    bollinger_up = moving_avg + std**bollinger_vol_exp * sigma 
    bollinger_down = moving_avg - std**bollinger_vol_exp * sigma 
    return bollinger_up, bollinger_down

def get_bands(zscore, sigma=entry_sigma):
    if band_type == 'bollinger':
        upper_band, lower_band = get_bollinger_bands(zscore)
    # elif band_type == 'keltner':
    #     upper_band, lower_band = get_keltner_channels()
    return upper_band, lower_band

def get_moving_avg(prices, override=None):
    prices_df = pd.DataFrame(prices)
    if not override: 
        type = average_type
    else: 
        type = override
    if type =='sma':
        ma_df = prices_df.rolling(lookback).mean()
    elif type == 'ema':
        ma_df = prices_df.ewm(span=lookback).mean()
    return ma_df

In [3]:
ticker1 = 'ETH/BUSD'
ticker2 = 'DOGE/BUSD'
bar_size_lookback = '5m'
duration_lookback = 250
data1 = get_prices_crypto_websocket(ticker1, bar_size_lookback, duration_lookback, ohclv=True, return_all_data=True)
data2 = get_prices_crypto_websocket(ticker2, bar_size_lookback, duration_lookback, ohclv=True, return_all_data=True)



In [35]:
global global_data,hedge_ratio, duration_lookback, bar_size_lookback
duration = duration_lookback
update, replace_df = False, False

latest_time_pulled = get_prices_crypto_websocket(ticker1, bar_size_lookback, 2, ohclv=False, return_all_data=True) #*** need to change the time of the last bar from binace

#latest_time_pulled = get_prices_crypto(ticker1, bar_size_lookback, 2, return_all_data=True)
delta_time = int(abs((latest_time_pulled[0][0] - latest_time_pulled[1][0])/1000)) #in seconds

# #get the latest time in the df
latest_df_time = global_data['Time'].iloc[0]
print(latest_df_time)
print(latest_time_pulled[0][0])
differential = latest_time_pulled[0][0]- latest_df_time
print(differential)
if differential > 0: 
    delta_bars = ((latest_time_pulled[0][0] - latest_df_time)/(delta_time*1000))*1.2
    if delta_bars > len(global_data): 
        print('Df replace!')
        print(global_data)
        replace_df = True #update entire dataframe
    else: 
        duration = delta_bars
elif differential == 0: 
    update = True
    duration = 1
            
data1, data2 = data1[::-1], data2[::-1] #needed to reverse list

data1 = data1[:global_data_max_size]
data2 = data2[:global_data_max_size]

times = [data1[i][0] for i in range(len(data1))]

data1_close = [data1[i][4] for i in range(len(data1))]
data2_close = [data2[i][4] for i in range(len(data2))]

m,b = np.polyfit(data2_close,data1_close,1)
hedge_ratio = m

data1_low = [data1[i][3] for i in range(len(data1))]
data2_low = [data2[i][3] for i in range(len(data2))]

data1_high = [data1[i][2] for i in range(len(data1))]
data2_high = [data2[i][2] for i in range(len(data2))]

spread = get_spread(prices_a = data1_close, prices_b = data2_close)
zscore, upper_band, lower_band, moving_avg  = get_data(spread)

#times, data1_close, data1_low, data1_high, data2_close, data2_low, data2_high, spread, zscore, upper_band, lower_band, moving_avg = times[::-1], data1_close[::-1], data1_low[::-1], data1_high[::-1], data2_close[::-1], data2_low[::-1], data2_high[::-1], spread[::-1], zscore[::-1], upper_band[::-1], lower_band[::-1], moving_avg[::-1]

df = pd.DataFrame({'Time':times, 'Ticker 1 Price':data1_close,'Ticker 1 Low':data1_low, 'Ticker 1 High':data1_high,'Ticker 2 Price':data2_close, 'Ticker 2 Low':data2_low, 'Ticker 2 High':data2_high,'Spread': spread,'Zscore': zscore, 'Upper Band': upper_band, 'Lower Band': lower_band, 'Moving Average': moving_avg})
print(global_data)
# global_data=df
# print(delta_time)
# print(latest_df_time)
# print(latest_time_pulled[-1][0])

print(delta_time)
print(delta_bars)


1666718400000.0
1666719000000.0
600000.0
             Time  Ticker 1 Price  Ticker 1 Low  Ticker 1 High  \
0    1.666718e+12         1454.85       1451.80        1456.10   
1    1.666718e+12         1452.86       1450.40        1466.84   
2    1.666718e+12         1466.11       1459.97        1479.52   
3    1.666718e+12         1476.11       1468.58        1488.79   
4    1.666717e+12         1480.98       1426.23        1773.40   
..            ...             ...           ...            ...   
245  1.666645e+12         1350.13       1349.79        1351.20   
246  1.666645e+12         1350.01       1348.15        1351.32   
247  1.666644e+12         1348.80       1348.15        1351.37   
248  1.666644e+12         1350.03       1346.49        1350.29   
249  1.666644e+12         1347.23       1346.02        1349.09   

     Ticker 2 Price  Ticker 2 Low  Ticker 2 High        Spread    Zscore  \
0           0.06163       0.06157        0.06198  95890.815551       NaN   
1           0.